In [2]:
import re
import pandas as pd
import numpy as np
import nltk
# import string
# import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords



def openfile(path):
    f=open(path)
    text=f.read()
    f.close() 
    all_articles=text.split("\n\n")
#     print(("len of article: ",len(all_articles)))
    return all_articles



def get_key_abs(all_articles,keywords_abstracts):
    for art in all_articles:
        if ("\nDE " in art) and ("\nTI " in art) and ("\nID " in art):
            DEs = re.findall(r'\sDE[ ].*[\s\w;-]*\sID', art)
            TIs = re.findall(r'\sTI[ ].*[\s\w:\-]+', art)
            if len(DEs)==1 and len(TIs)==1:
                sentence = ""
                for DE in DEs:
                    DE=DE.split("ID")[0]
                    DE=DE.split("DE")[1].strip().lower()
                    DE=re.sub(r"\s+", ' ',DE)
                    sentence = sentence + DE + ". "
                for TI in TIs:
                    TI=TI.split("SO")[0]
                    TI=TI.split("TI")[1].strip().lower()
                    TI=re.sub(r"\s+", ' ',TI)
                    sentence = sentence + TI
                    sentence=re.sub('[^a-zA-Z;.,-]',' ',sentence)
                    sentence=re.sub('\s+',' ',sentence)                
                keywords_abstracts.append(sentence)
                


if __name__ == "__main__":
    keywords_abstracts = []
    
    for i in range(1,42):
        path = "../data/biomaterial/savedrecs%d.txt"%i
        all_articles = openfile(path)
        get_key_abs(all_articles,keywords_abstracts)
        print(("len of total: ",len(keywords_abstracts)))
        
    for i in range(1,48):
        path = "../data/biomedical_material/savedrecs%d.txt"%i
        all_articles = openfile(path)
        get_key_abs(all_articles,keywords_abstracts)
        print(("len of total: ",len(keywords_abstracts)))
    
print(keywords_abstracts[0])
print(keywords_abstracts[1])

('len of total: ', 419)
('len of total: ', 831)
('len of total: ', 1214)
('len of total: ', 1604)
('len of total: ', 1966)
('len of total: ', 2322)
('len of total: ', 2704)
('len of total: ', 3088)
('len of total: ', 3469)
('len of total: ', 3752)
('len of total: ', 4122)
('len of total: ', 4498)
('len of total: ', 4765)
('len of total: ', 5129)
('len of total: ', 5482)
('len of total: ', 5836)
('len of total: ', 6164)
('len of total: ', 6553)
('len of total: ', 6881)
('len of total: ', 7202)
('len of total: ', 7564)
('len of total: ', 7883)
('len of total: ', 8248)
('len of total: ', 8584)
('len of total: ', 8933)
('len of total: ', 9287)
('len of total: ', 9639)
('len of total: ', 9971)
('len of total: ', 10291)
('len of total: ', 10634)
('len of total: ', 10947)
('len of total: ', 11268)
('len of total: ', 11593)
('len of total: ', 11911)
('len of total: ', 12234)
('len of total: ', 12536)
('len of total: ', 12838)
('len of total: ', 13141)
('len of total: ', 13403)
('len of total: 

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# LDA can only use raw term counts for LDA 
tf_vectorizer = CountVectorizer(max_df=0.90, \
                min_df=50, stop_words='english')
tf = tf_vectorizer.fit_transform(keywords_abstracts)

# each feature is a word (bag of words)
# get_feature_names() gives all words
tf_feature_names = tf_vectorizer.get_feature_names()

print(tf_feature_names[1000:1010])
print(tf.shape)
# print(tf)

# split dataset into train (90%) and test sets (10%)
# the test sets will be used to evaluate proplexity of topic modeling
X_train, X_test = train_test_split(\
                tf, test_size=0.1, random_state=0)

['protein', 'proteins', 'prototyping', 'pseudomonas', 'pulp', 'pulsed', 'pure', 'purification', 'pva', 'quality']
(28256, 1341)


In [4]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 24            # how to estimate the num of topic?

# Run LDA. For details, check
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation.perplexity

# max_iter control the number of iterations 
# evaluate_every determines how often the perplexity is calculated
# n_jobs is the number of parallel threads
lda = LatentDirichletAllocation(n_components=num_topics, \
                                max_iter=12,verbose=1,
                                evaluate_every=1, n_jobs=1,
                                random_state=0).fit(X_train)

iteration: 1 of max_iter: 12, perplexity: 965.4748
iteration: 2 of max_iter: 12, perplexity: 711.6839
iteration: 3 of max_iter: 12, perplexity: 591.1724
iteration: 4 of max_iter: 12, perplexity: 533.4799
iteration: 5 of max_iter: 12, perplexity: 503.8434
iteration: 6 of max_iter: 12, perplexity: 486.4711
iteration: 7 of max_iter: 12, perplexity: 475.7665
iteration: 8 of max_iter: 12, perplexity: 469.1166
iteration: 9 of max_iter: 12, perplexity: 464.7383
iteration: 10 of max_iter: 12, perplexity: 461.9114
iteration: 11 of max_iter: 12, perplexity: 460.1442
iteration: 12 of max_iter: 12, perplexity: 459.0391


In [5]:
num_top_words=10

# lda.components_ returns a KxN matrix
# for word distribution in each topic.
# Each row consists of 
# probability (counts) of each word in the feature space

for topic_idx, topic in enumerate(lda.components_):
    print ("Topic %d:" % (topic_idx))
    # print out top 20 words per topic 
    words=[tf_feature_names[i] for i in topic.argsort()[::-1][0:num_top_words]]
    print(words)
    print("")

Topic 0:
['laser', 'corrosion', 'magnesium', 'coating', 'alloy', 'steel', 'stainless', 'mg', 'polyurethane', 'electrochemical']

Topic 1:
['tissue', 'engineering', 'scaffolds', 'scaffold', 'matrix', 'regeneration', 'biomaterial', 'biomaterials', 'extracellular', 'medicine']

Topic 2:
['materials', 'biomedical', 'review', 'porous', 'design', 'applications', 'material', 'printing', 'based', 'manufacturing']

Topic 3:
['plasma', 'adsorption', 'deposition', 'coatings', 'polyethylene', 'grafting', 'metal', 'biomaterial', 'gamma', 'albumin']

Topic 4:
['hydroxyapatite', 'synthesis', 'organic', 'characterization', 'composite', 'quantum', 'microwave', 'inorganic', 'solid', 'method']

Topic 5:
['surface', 'modification', 'protein', 'surfaces', 'blood', 'plasma', 'adsorption', 'compatibility', 'biomaterial', 'titanium']

Topic 6:
['shape', 'memory', 'nerve', 'analysis', 'element', 'finite', 'pressure', 'modeling', 'non', 'model']

Topic 7:
['chitosan', 'wound', 'healing', 'gelatin', 'alginate', 